In [3]:
# some basic looking at the data:
# 1. distributions of the p values and slope values (both individual and group)
# 2. plot the slope vs the variance

In [4]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
import nilearn
import json
import datetime
import pickle
import seaborn as sns
import gc
import psutil
import math
import scipy.stats as stats
from matplotlib.patches import Patch
from nilearn import plotting
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.second_level import SecondLevelModel
from nilearn.glm import threshold_stats_img
from nilearn.image import concat_imgs, mean_img, index_img
from nilearn.reporting import make_glm_report
from nilearn import masking, image
from nilearn import datasets
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from collections import defaultdict
from nilearn.maskers import NiftiLabelsMasker
from nilearn.plotting.find_cuts import find_cut_slices

# Import shared utilities and configuration
# need to do it this way because in a sub-directory (later turn config and utils into part of a package)
from utils import (
    TASKS, CONTRASTS, SUBJECTS, SESSIONS, ENCOUNTERS,
    build_first_level_contrast_map_path, is_valid_contrast_map, clean_z_map_data,
    convert_to_regular_dict, create_smor_atlas,load_smor_atlas, load_schaefer_atlas, cleanup_memory
)
from config import BASE_DIR, OUTPUT_DIRS

In [5]:
compiled_req_contrasts = []
for task in TASKS:
    for contrast in CONTRASTS[task]:
        if (contrast not in compiled_req_contrasts):
            compiled_req_contrasts.append(contrast)

In [6]:
# smorgasbord stuff
SMORG_PARCELLATED_DIR = OUTPUT_DIRS["smor"]
smor_files = {'mean':f'discovery_parcel_indiv_mean_updated'}
smor_date_updated = '0111'
indices = [1,2,3]
# get smorgasbord atlas
smorgasbord_atlas = load_smor_atlas()
SMORG_IMG = smorgasbord_atlas.maps
SMORG_DATA = SMORG_IMG.get_fdata()


# atlas
req_atlas = "smor"
main_dir = SMORG_PARCELLATED_DIR
main_files = smor_files
date_updated = smor_date_updated
atlas_obj = smorgasbord_atlas

Loading Smorgasbord atlas...
Atlas loaded with 429 regions
Atlas shape: (193, 229, 193)


In [7]:
# file loading
file_type = "default" # can change to "default" to make this code use betas
output_ending = "_betas"
if (file_type == "z"):
    output_ending = "_z_scored"
        
# Load mean parcel data from multiple files
mean_filename = f"{main_dir}/{main_files['mean']}_{date_updated}"

In [8]:
# load the individual trajectories and data
indiv_dict_file = f'{mean_filename}{output_ending}_indiv_slopes.pkl'
with open(indiv_dict_file, 'rb') as f:
    indiv_parcel_traj_results = pickle.load(f)
    
print(f"\nTotal subjects loaded: {len(indiv_parcel_traj_results)}")
print(f"Atlas: {req_atlas} ({len(atlas_obj.labels)} regions)")


Total subjects loaded: 5
Atlas: smor (429 regions)


In [9]:
# load the group averaged trajectories and data
group_averaged_dict_file = f'{mean_filename}{output_ending}_averaged.pkl'
with open(group_averaged_dict_file, 'rb') as f:
    avg_parcel_traj_results = pickle.load(f)

# plot individual distributions of p vals and slopes

In [10]:
def plot_within_subject_effects(parcel_traj_results, subject, task, contrast, 
                                save_dir='figures/within_subject'):
    """
    Plot distribution of slopes across parcels for a SINGLE subject for a specific task/contrast.
    
    Parameters:
    -----------
    parcel_traj_results : dict
        Individual results [subject][task][contrast][parcel]
    subject : str
        Subject ID to plot
    task : str
        Specific task
    contrast : str
        Specific contrast
    save_dir : str
        Directory to save figure
    """
    
    os.makedirs(save_dir, exist_ok=True)
    
    # ==========================================
    # COLLECT DATA FOR THIS SUBJECT
    # ==========================================
    
    all_slopes = []
    all_p_values = []
    
    plot_title = f"{subject} | {task} | {contrast}"
    filename = f"within_subj_{subject}_{task}_{contrast}"
    
    # Collect all parcel-level values for this subject/task/contrast
    for parcel, stats_dict in parcel_traj_results[subject][task][contrast].items():
        all_slopes.append(stats_dict['beta_slope'])
        all_p_values.append(stats_dict['p_value'])
    
    # Convert to arrays and remove NaNs
    all_slopes = np.array(all_slopes)
    all_p_values = np.array(all_p_values)
    
    valid = ~np.isnan(all_p_values)
    
    print(f"\n{'='*60}")
    print(f"WITHIN-SUBJECT SUMMARY: {plot_title}")
    print(f"{'='*60}")
    print(f"Total parcels: {np.sum(valid)}")

    # plot figure
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle(f'Within-Subject Practice Effects: {plot_title}', 
                 fontsize=14, fontweight='bold')
    
    # SLOPE DISTRIBUTION    
    ax1 = axes[0]
    
    # Separate significant and non-significant
    sig_mask = all_p_values[valid] < 0.05
    nonsig_mask = all_p_values[valid] >= 0.05
    
    # Plot both distributions with different colors
    ax1.hist(all_slopes[valid][nonsig_mask], bins=60, edgecolor='black', 
             alpha=0.5, color='gray', label='Not significant')
    ax1.hist(all_slopes[valid][sig_mask], bins=60, edgecolor='black', 
             alpha=0.7, color='red', label='Significant (p<0.05)')
    
    ax1.axvline(x=0, color='black', linestyle='-', linewidth=2, label='Zero')
    ax1.axvline(x=np.median(all_slopes[valid]), color='blue', 
                linestyle='--', linewidth=2, label=f'Median={np.median(all_slopes[valid]):.4f}')
    
    ax1.set_xlabel('Slope (Practice Effect)', fontsize=12)
    ax1.set_ylabel('Frequency', fontsize=12)
    ax1.set_title('Distribution Across Parcels', fontsize=13, fontweight='bold')
    ax1.legend()
    
    n_positive = np.sum(all_slopes[valid] > 0)
    n_total = np.sum(valid)
    n_sig = np.sum(sig_mask)
    n_sig_positive = np.sum((all_slopes[valid] > 0) & sig_mask)
    n_sig_negative = np.sum((all_slopes[valid] < 0) & sig_mask)
    
    ax1.text(0.02, 0.98, 
             f'N parcels: {n_total}\n'
             f'Mean: {np.mean(all_slopes[valid]):.5f}\n'
             f'Median: {np.median(all_slopes[valid]):.5f}\n'
             f'SD: {np.std(all_slopes[valid]):.5f}\n'
             f'\n'
             f'Significant: {n_sig} ({100*n_sig/n_total:.1f}%)\n'
             f'  Positive: {n_sig_positive}\n'
             f'  Negative: {n_sig_negative}\n'
             f'\n'
             f'All positive: {n_positive} ({100*n_positive/n_total:.1f}%)',
             transform=ax1.transAxes, ha='left', va='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.9),
             fontsize=9)
    
    # VOLCANO PLOT

    ax2 = axes[1]
    
    # Plot non-significant first (so significant appear on top)
    ax2.scatter(all_slopes[valid][nonsig_mask], 
                -np.log10(all_p_values[valid][nonsig_mask]),
                c='gray', alpha=0.3, s=10, label='Not sig')
    ax2.scatter(all_slopes[valid][sig_mask], 
                -np.log10(all_p_values[valid][sig_mask]),
                c='red', alpha=0.6, s=10, label='p<0.05')
    
    # Add reference lines
    ax2.axhline(y=-np.log10(0.05), color='red', linestyle='--', 
                linewidth=1.5, alpha=0.7, label='p=0.05')
    ax2.axvline(x=0, color='black', linestyle='-', linewidth=1)
    
    ax2.set_xlabel('Slope (Practice Effect)', fontsize=12)
    ax2.set_ylabel('-log10(p-value)', fontsize=12)
    ax2.set_title('Effect Size vs Significance', fontsize=13, fontweight='bold')
    ax2.legend(loc='upper right')
    
    plt.tight_layout()
    
    # Save
    save_path = os.path.join(save_dir, f'{filename}.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Figure saved: {save_path}")
    
    plt.show()
    
    # ==========================================
    # PRINT STATISTICS
    # ==========================================
    
    print(f"\nEFFECT SIZE STATISTICS:")
    print(f"  Mean slope: {np.mean(all_slopes[valid]):.6f}")
    print(f"  Median slope: {np.median(all_slopes[valid]):.6f}")
    print(f"  SD slope: {np.std(all_slopes[valid]):.6f}")
    print(f"  Range: [{np.min(all_slopes[valid]):.6f}, {np.max(all_slopes[valid]):.6f}]")
    print(f"\nDIRECTION:")
    print(f"  Positive slopes: {n_positive} / {n_total} ({100*n_positive/n_total:.1f}%)")
    print(f"  Negative slopes: {n_total - n_positive} / {n_total} ({100*(n_total-n_positive)/n_total:.1f}%)")
    print(f"\nSIGNIFICANCE:")
    print(f"  p < 0.05: {n_sig} / {n_total} ({100*n_sig/n_total:.1f}%)")
    print(f"    Positive & sig: {n_sig_positive}")
    print(f"    Negative & sig: {n_sig_negative}")
    
    return fig

In [11]:
def plot_pooled_effects(parcel_traj_results, task, contrast, 
                       save_dir='figures/pooled'):
    """
    Plot distribution of ALL subject×parcel slopes for a specific task/contrast.
    """
    os.makedirs(save_dir, exist_ok=True)
    
    all_slopes = []
    all_p_values = []
    
    plot_title = f"{task} | {contrast} (All Subjects)"
    filename = f"pooled_{task}_{contrast}"
    
    # Collect all subject×parcel values
    for subj in parcel_traj_results.keys():
        for parcel, stats_dict in parcel_traj_results[subj][task][contrast].items():
            all_slopes.append(stats_dict['beta_slope'])
            all_p_values.append(stats_dict['p_value'])
    
    all_slopes = np.array(all_slopes)
    all_p_values = np.array(all_p_values)
    valid = ~np.isnan(all_p_values)
    
    print(f"\n{'='*60}")
    print(f"POOLED: {plot_title}")
    print(f"Total: {np.sum(valid)}")
    print(f"{'='*60}")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle(f'Pooled Practice Effects: {plot_title}', fontsize=14, fontweight='bold')
    
    # SLOPE DISTRIBUTION
    ax1 = axes[0]
    sig_mask = all_p_values[valid] < 0.05
    nonsig_mask = ~sig_mask
    
    ax1.hist(all_slopes[valid][nonsig_mask], bins=60, edgecolor='black', 
             alpha=0.5, color='gray', label='Not significant')
    ax1.hist(all_slopes[valid][sig_mask], bins=60, edgecolor='black', 
             alpha=0.7, color='red', label='Significant (p<0.05)')
    ax1.axvline(x=0, color='black', linestyle='-', linewidth=2, label='Zero')
    ax1.axvline(x=np.median(all_slopes[valid]), color='blue', 
                linestyle='--', linewidth=2, label=f'Median={np.median(all_slopes[valid]):.4f}')
    ax1.set_xlabel('Slope (Practice Effect)', fontsize=12)
    ax1.set_ylabel('Frequency', fontsize=12)
    ax1.set_title('Distribution Across All Subjects & Parcels', fontsize=13, fontweight='bold')
    ax1.legend()
    
    # Stats box
    n_total = np.sum(valid)
    n_sig = np.sum(sig_mask)
    n_positive = np.sum(all_slopes[valid] > 0)
    n_sig_pos = np.sum((all_slopes[valid] > 0) & sig_mask)
    n_sig_neg = np.sum((all_slopes[valid] < 0) & sig_mask)
    
    ax1.text(0.02, 0.98, 
             f'N: {n_total}\n'
             f'Mean: {np.mean(all_slopes[valid]):.5f}\n'
             f'Median: {np.median(all_slopes[valid]):.5f}\n'
             f'SD: {np.std(all_slopes[valid]):.5f}\n'
             f'\nSig: {n_sig} ({100*n_sig/n_total:.1f}%)\n'
             f'  Pos: {n_sig_pos}\n'
             f'  Neg: {n_sig_neg}\n'
             f'\nAll pos: {n_positive} ({100*n_positive/n_total:.1f}%)',
             transform=ax1.transAxes, ha='left', va='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.9),
             fontsize=9)
    
    # VOLCANO PLOT
    ax2 = axes[1]
    ax2.scatter(all_slopes[valid][nonsig_mask], -np.log10(all_p_values[valid][nonsig_mask]),
                c='gray', alpha=0.3, s=10, label='Not sig')
    ax2.scatter(all_slopes[valid][sig_mask], -np.log10(all_p_values[valid][sig_mask]),
                c='red', alpha=0.6, s=10, label='p<0.05')
    ax2.axhline(y=-np.log10(0.05), color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='p=0.05')
    ax2.axvline(x=0, color='black', linestyle='-', linewidth=1)
    ax2.set_xlabel('Slope (Practice Effect)', fontsize=12)
    ax2.set_ylabel('-log10(p-value)', fontsize=12)
    ax2.set_title('Effect Size vs Significance', fontsize=13, fontweight='bold')
    ax2.legend(loc='upper right')
    
    plt.tight_layout()
    save_path = os.path.join(save_dir, f'{filename}.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Saved: {save_path}\n")
    plt.show()
    
    print(f"Mean: {np.mean(all_slopes[valid]):.6f}")
    print(f"Median: {np.median(all_slopes[valid]):.6f}")
    print(f"Sig: {n_sig}/{n_total} ({100*n_sig/n_total:.1f}%)")
    print(f"Pos: {n_positive}/{n_total} ({100*n_positive/n_total:.1f}%)\n")
    
    return fig

In [ ]:
# Get all unique task/contrast combinations from the data
all_tasks = set()
all_contrasts = {}

first_subj = list(indiv_parcel_traj_results.keys())[0]
for task in indiv_parcel_traj_results[first_subj].keys():
    all_tasks.add(task)
    all_contrasts[task] = list(indiv_parcel_traj_results[first_subj][task].keys())

print("Available tasks:", sorted(all_tasks))
for task in sorted(all_tasks):
    print(f"  {task}: {all_contrasts[task]}")

# WITHIN-SUBJECT EFFECTS
# (One plot per subject per task/contrast)

# Loop through each subject
for subject in indiv_parcel_traj_results.keys():
    print(f"\n--- Subject: {subject} ---")
    
    # Loop through each task
    for task in sorted(all_tasks):
        # Loop through each contrast in this task
        for contrast in sorted(all_contrasts[task]):
            print(f"\nPlotting: {subject} | {task} | {contrast}")
            
            fig = plot_within_subject_effects(
                indiv_parcel_traj_results, 
                subject=subject,
                task=task, 
                contrast=contrast
            )
            plt.close(fig)  # Close to avoid memory issues


# POOLED EFFECTS
# (One plot per task/contrast, pooling all subjects)

# Loop through each task
for task in sorted(all_tasks):
    # Loop through each contrast in this task
    for contrast in sorted(all_contrasts[task]):
        print(f"\nPlotting pooled: {task} | {contrast}")
        
        fig = plot_pooled_effects(
            indiv_parcel_traj_results,
            task=task, 
            contrast=contrast
        )
        plt.close(fig)  # Close to avoid memory issues

# plot distributions of averaged slopes

In [16]:
def plot_group_averaged_effects(avg_parcel_traj_results, task, contrast, 
                                save_dir='figures/group_averaged'):
    """
    Plot distribution of group-averaged slopes across parcels for a specific task/contrast.
    
    Parameters:
    -----------
    avg_parcel_traj_results : dict
        Group-averaged results [task][contrast][parcel]
    task : str
        Specific task
    contrast : str
        Specific contrast
    save_dir : str
        Directory to save figure
    """
    os.makedirs(save_dir, exist_ok=True)
    
    all_slopes = []
    all_p_values = []
    
    plot_title = f"{task} | {contrast} (Group Average)"
    filename = f"group_avg_{task}_{contrast}"
    
    # Collect all parcel values
    for parcel, stats_dict in avg_parcel_traj_results[task][contrast].items():
        all_slopes.append(stats_dict['slope_mean'])
        all_p_values.append(stats_dict['group_p_value'])
    
    all_slopes = np.array(all_slopes)
    all_p_values = np.array(all_p_values)
    valid = ~np.isnan(all_p_values)
    
    print(f"\n{'='*60}")
    print(f"GROUP AVERAGE: {plot_title}")
    print(f"Total parcels: {np.sum(valid)}")
    print(f"{'='*60}")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle(f'Group-Averaged Practice Effects: {plot_title}', fontsize=14, fontweight='bold')
    
    # SLOPE DISTRIBUTION
    ax1 = axes[0]
    sig_mask = all_p_values[valid] < 0.05
    nonsig_mask = ~sig_mask
    
    ax1.hist(all_slopes[valid][nonsig_mask], bins=60, edgecolor='black', 
             alpha=0.5, color='gray', label='Not significant')
    ax1.hist(all_slopes[valid][sig_mask], bins=60, edgecolor='black', 
             alpha=0.7, color='red', label='Significant (p<0.05)')
    ax1.axvline(x=0, color='black', linestyle='-', linewidth=2, label='Zero')
    ax1.axvline(x=np.median(all_slopes[valid]), color='blue', 
                linestyle='--', linewidth=2, label=f'Median={np.median(all_slopes[valid]):.4f}')
    ax1.set_xlabel('Slope (Practice Effect)', fontsize=12)
    ax1.set_ylabel('Frequency', fontsize=12)
    ax1.set_title('Distribution Across Parcels', fontsize=13, fontweight='bold')
    ax1.legend()
    
    # Stats box
    n_total = np.sum(valid)
    n_sig = np.sum(sig_mask)
    n_positive = np.sum(all_slopes[valid] > 0)
    n_sig_pos = np.sum((all_slopes[valid] > 0) & sig_mask)
    n_sig_neg = np.sum((all_slopes[valid] < 0) & sig_mask)
    
    ax1.text(0.02, 0.98, 
             f'N parcels: {n_total}\n'
             f'Mean: {np.mean(all_slopes[valid]):.5f}\n'
             f'Median: {np.median(all_slopes[valid]):.5f}\n'
             f'SD: {np.std(all_slopes[valid]):.5f}\n'
             f'\nSig: {n_sig} ({100*n_sig/n_total:.1f}%)\n'
             f'  Pos: {n_sig_pos}\n'
             f'  Neg: {n_sig_neg}\n'
             f'\nAll pos: {n_positive} ({100*n_positive/n_total:.1f}%)',
             transform=ax1.transAxes, ha='left', va='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.9),
             fontsize=9)
    
    # VOLCANO PLOT
    ax2 = axes[1]
    ax2.scatter(all_slopes[valid][nonsig_mask], -np.log10(all_p_values[valid][nonsig_mask]),
                c='gray', alpha=0.3, s=10, label='Not sig')
    ax2.scatter(all_slopes[valid][sig_mask], -np.log10(all_p_values[valid][sig_mask]),
                c='red', alpha=0.6, s=10, label='p<0.05')
    ax2.axhline(y=-np.log10(0.05), color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='p=0.05')
    ax2.axvline(x=0, color='black', linestyle='-', linewidth=1)
    ax2.set_xlabel('Slope (Practice Effect)', fontsize=12)
    ax2.set_ylabel('-log10(p-value)', fontsize=12)
    ax2.set_title('Effect Size vs Significance', fontsize=13, fontweight='bold')
    ax2.legend(loc='upper right')
    
    plt.tight_layout()
    save_path = os.path.join(save_dir, f'{filename}.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Saved: {save_path}\n")
    plt.show()
    
    print(f"Mean: {np.mean(all_slopes[valid]):.6f}")
    print(f"Median: {np.median(all_slopes[valid]):.6f}")
    print(f"Sig: {n_sig}/{n_total} ({100*n_sig/n_total:.1f}%)")
    print(f"Pos: {n_positive}/{n_total} ({100*n_positive/n_total:.1f}%)\n")
    
    return fig

In [ ]:
all_tasks_avg = list(avg_parcel_traj_results.keys())
all_contrasts_avg = {}
for task in all_tasks_avg:
    all_contrasts_avg[task] = list(avg_parcel_traj_results[task].keys())

print("Available in group average:")
for task in sorted(all_tasks_avg):
    print(f"  {task}: {all_contrasts_avg[task]}")

# Plot all task/contrast combinations
print("PLOTTING GROUP-AVERAGED EFFECTS")

for task in sorted(all_tasks_avg):
    for contrast in sorted(all_contrasts_avg[task]):
        print(f"\nPlotting: {task} | {contrast}")
        
        fig = plot_group_averaged_effects(
            avg_parcel_traj_results,
            task=task, 
            contrast=contrast
        )
        if fig is not None:
            plt.close(fig)

In [22]:
def plot_group_averaged_effects(avg_parcel_traj_results, task, contrast, 
                                save_dir='figures/group_averaged'):
    """
    Plot distribution of group-averaged slopes across parcels for a specific task/contrast.
    Colors points by the proportion of subjects showing significant effects.
    
    Parameters:
    -----------
    avg_parcel_traj_results : dict
        Group-averaged results [task][contrast][parcel]
    task : str
        Specific task
    contrast : str
        Specific contrast
    save_dir : str
        Directory to save figure
    """
    import os
    os.makedirs(save_dir, exist_ok=True)
    
    all_slopes = []
    all_p_values = []
    all_prop_sig = []
    
    plot_title = f"{task} | {contrast} (Group Average)"
    filename = f"group_avg_{task}_{contrast}"
    
    # Collect all parcel values
    for parcel, stats_dict in avg_parcel_traj_results[task][contrast].items():
        all_slopes.append(stats_dict['slope_mean'])
        all_p_values.append(stats_dict['group_p_value'])
        all_prop_sig.append(stats_dict['individual_p_significant_proportion'])
    
    all_slopes = np.array(all_slopes)
    all_p_values = np.array(all_p_values)
    all_prop_sig = np.array(all_prop_sig)
    valid = ~np.isnan(all_p_values)
    
    # Check if we have any valid data
    if np.sum(valid) == 0:
        print(f"\n{'='*60}")
        print(f"WARNING: No valid data for {plot_title}")
        print(f"All p-values are NaN - skipping plot")
        print(f"{'='*60}\n")
        return None
    
    print(f"\n{'='*60}")
    print(f"GROUP AVERAGE: {plot_title}")
    print(f"Total parcels: {np.sum(valid)}")
    print(f"{'='*60}")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle(f'Group-Averaged Practice Effects: {plot_title}', fontsize=14, fontweight='bold')
    
    # SLOPE DISTRIBUTION
    ax1 = axes[0]
    sig_mask = all_p_values[valid] < 0.05
    nonsig_mask = ~sig_mask
    
    # Only plot if there's data in each category
    if np.sum(nonsig_mask) > 0:
        ax1.hist(all_slopes[valid][nonsig_mask], bins=60, edgecolor='black', 
                 alpha=0.5, color='gray', label='Not significant')
    if np.sum(sig_mask) > 0:
        ax1.hist(all_slopes[valid][sig_mask], bins=60, edgecolor='black', 
                 alpha=0.7, color='red', label='Significant (p<0.05)')
    
    ax1.axvline(x=0, color='black', linestyle='-', linewidth=2, label='Zero')
    ax1.axvline(x=np.median(all_slopes[valid]), color='blue', 
                linestyle='--', linewidth=2, label=f'Median={np.median(all_slopes[valid]):.4f}')
    ax1.set_xlabel('Slope (Practice Effect)', fontsize=12)
    ax1.set_ylabel('Frequency', fontsize=12)
    ax1.set_title('Distribution Across Parcels', fontsize=13, fontweight='bold')
    ax1.legend()
    
    # Stats box
    n_total = np.sum(valid)
    n_sig = np.sum(sig_mask)
    n_positive = np.sum(all_slopes[valid] > 0)
    n_sig_pos = np.sum((all_slopes[valid] > 0) & sig_mask)
    n_sig_neg = np.sum((all_slopes[valid] < 0) & sig_mask)
    
    ax1.text(0.02, 0.98, 
             f'N parcels: {n_total}\n'
             f'Mean: {np.mean(all_slopes[valid]):.5f}\n'
             f'Median: {np.median(all_slopes[valid]):.5f}\n'
             f'SD: {np.std(all_slopes[valid]):.5f}\n'
             f'\nSig: {n_sig} ({100*n_sig/n_total:.1f}%)\n'
             f'  Pos: {n_sig_pos}\n'
             f'  Neg: {n_sig_neg}\n'
             f'\nAll pos: {n_positive} ({100*n_positive/n_total:.1f}%)',
             transform=ax1.transAxes, ha='left', va='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.9),
             fontsize=9)
    
    # VOLCANO PLOT - COLORED BY PROPORTION SIGNIFICANT
    ax2 = axes[1]
    
    # Create scatter plot colored by proportion of subjects significant
    scatter = ax2.scatter(all_slopes[valid], 
                         -np.log10(all_p_values[valid]),
                         c=all_prop_sig[valid], 
                         cmap='RdYlGn',  # Red (0) to Yellow (0.5) to Green (1)
                         vmin=0, vmax=1,
                         s=20, alpha=0.7, edgecolors='black', linewidth=0.5)
    
    # Add colorbar
    cbar = plt.colorbar(scatter, ax=ax2)
    cbar.set_label('Proportion of Subjects\nwith p<0.05', fontsize=10)
    
    # Add reference lines
    ax2.axhline(y=-np.log10(0.05), color='red', linestyle='--', 
                linewidth=1.5, alpha=0.7, label='Group p=0.05')
    ax2.axvline(x=0, color='black', linestyle='-', linewidth=1)
    
    ax2.set_xlabel('Slope (Practice Effect)', fontsize=12)
    ax2.set_ylabel('-log10(p-value)', fontsize=12)
    ax2.set_title('Effect Size vs Significance\n(colored by individual consistency)', 
                  fontsize=13, fontweight='bold')
    ax2.legend(loc='upper right')
    
    plt.tight_layout()
    save_path = os.path.join(save_dir, f'{filename}.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Saved at {save_path}\n")
    plt.show()
    
    print(f"\nCONSISTENCY ACROSS SUBJECTS:")
    print(f"  Mean prop sig: {np.mean(all_prop_sig[valid]):.3f}")
    print(f"  Median prop sig: {np.median(all_prop_sig[valid]):.3f}")
    print(f"  Parcels with >50% subjects sig: {np.sum(all_prop_sig[valid] > 0.5)} ({100*np.sum(all_prop_sig[valid] > 0.5)/n_total:.1f}%)")
    print(f"  Parcels with 100% subjects sig: {np.sum(all_prop_sig[valid] == 1.0)} ({100*np.sum(all_prop_sig[valid] == 1.0)/n_total:.1f}%)")
    
    print(f"\nEFFECT SIZE STATISTICS:")
    print(f"  Mean: {np.mean(all_slopes[valid]):.6f}")
    print(f"  Median: {np.median(all_slopes[valid]):.6f}")
    print(f"  Sig: {n_sig}/{n_total} ({100*n_sig/n_total:.1f}%)")
    print(f"  Pos: {n_positive}/{n_total} ({100*n_positive/n_total:.1f}%)\n")
    
    return fig

In [ ]:
all_tasks_avg = list(avg_parcel_traj_results.keys())
all_contrasts_avg = {}
for task in all_tasks_avg:
    all_contrasts_avg[task] = list(avg_parcel_traj_results[task].keys())

print("Available in group average:")
for task in sorted(all_tasks_avg):
    print(f"  {task}: {all_contrasts_avg[task]}")

# Plot all task/contrast combinations
print("PLOTTING GROUP-AVERAGED EFFECTS")

for task in sorted(all_tasks_avg):
    for contrast in sorted(all_contrasts_avg[task]):
        print(f"\nPlotting: {task} | {contrast}")
        
        fig = plot_group_averaged_effects(
            avg_parcel_traj_results,
            task=task, 
            contrast=contrast
        )
        if fig is not None:
            plt.close(fig)

# slope vs variance (add to the old OHBM plots later)